In [261]:
import pandas as pd
import numpy as np
import seaborn as sns

# Imported Airbnb Calendar

In [260]:
df_calendar = pd.read_csv('calendar.csv.gz')
df_calendar

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,3395898,2020-05-30,f,"$5,000.00","$5,000.00",28.0,1125.0
1,5262640,2020-05-30,t,"$4,515.00","$4,515.00",3.0,1125.0
2,5262640,2020-05-31,t,"$4,515.00","$4,515.00",3.0,1125.0
3,5262640,2020-06-01,t,"$4,515.00","$4,515.00",3.0,1125.0
4,5262640,2020-06-02,t,"$4,515.00","$4,515.00",3.0,1125.0
...,...,...,...,...,...,...,...
5478911,43500050,2021-05-25,f,"$8,000.00","$8,000.00",2.0,300.0
5478912,43500050,2021-05-26,f,"$8,000.00","$8,000.00",2.0,300.0
5478913,43500050,2021-05-27,f,"$8,000.00","$8,000.00",2.0,300.0
5478914,43500050,2021-05-28,f,"$8,000.00","$8,000.00",2.0,300.0


In [262]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5478916 entries, 0 to 5478915
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   adjusted_price  object 
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 292.6+ MB


# Imported reviews

In [267]:
df_reviews = pd.read_csv('reviews.csv.gz')
df_reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,35303,810980,2011-12-28,1502908,Firuz,Miyuki's has been a great host. Helpful and ve...
1,35303,2467161,2012-10-01,350719,Jordan,Nice apartment in one of the best areas of Tok...
2,35303,3572628,2013-02-18,4917704,Aymeric,A very nice host and place. The location is in...
3,35303,3952097,2013-03-30,3243253,Blandine,Miyuki is the most smily face I met when I lan...
4,35303,4377509,2013-05-01,1536097,Kayleigh,The location's wonderful. It's close to Haraju...
...,...,...,...,...,...,...
398176,43415361,625690926,2020-05-23,208340666,Zensui,迅速な対応で､ｽﾑｰｽﾞに入室できました｡部屋も綺麗で使いやすかったです｡
398177,43430820,625472129,2020-05-21,230156826,Neko,ありがとうございました｡
398178,43430820,625839775,2020-05-24,346771114,Hikaru,ｺｽﾊﾟ抜群です!ﾙｰﾑﾗｲﾄがいい味出しています｡
398179,43430820,626062239,2020-05-25,345327445,みよし,ｺｽﾊﾟ高く､調理器具やｸﾛｰﾑｷｬｽﾄなどｴﾝﾀﾒが充実


In [279]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398181 entries, 0 to 398180
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     398181 non-null  int64 
 1   id             398181 non-null  int64 
 2   date           398181 non-null  object
 3   reviewer_id    398181 non-null  int64 
 4   reviewer_name  398179 non-null  object
 5   comments       397901 non-null  object
dtypes: int64(3), object(3)
memory usage: 18.2+ MB


# Imported Airbnb Listings

In [3]:
df_listings = pd.read_csv('listings.csv')
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15009 entries, 0 to 15008
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              15009 non-null  int64  
 1   name                            15009 non-null  object 
 2   host_id                         15009 non-null  int64  
 3   host_name                       15002 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   15009 non-null  object 
 6   latitude                        15009 non-null  float64
 7   longitude                       15009 non-null  float64
 8   room_type                       15009 non-null  object 
 9   price                           15009 non-null  int64  
 10  minimum_nights                  15009 non-null  int64  
 11  number_of_reviews               15009 non-null  int64  
 12  last_review                     

In [11]:
df_listings.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

## Explored Tokyo Airbnb data

In [73]:
counts_of_nights = df_listings['minimum_nights'].value_counts()

df_listings['room_type'].value_counts()

df_listings['host_name'].value_counts().head(10)

counts_of_neighborhoods = df_listings['neighbourhood'].value_counts()

df_listings['price'].value_counts()

counts_of_neighborhoods

Shinjuku Ku            2770
Taito Ku               2048
Toshima Ku             1553
Sumida Ku              1339
Shibuya Ku             1052
Chuo Ku                 610
Minato Ku               575
Ota Ku                  524
Nakano Ku               430
Arakawa Ku              393
Katsushika Ku           388
Setagaya Ku             379
Kita Ku                 357
Suginami Ku             335
Itabashi Ku             327
Edogawa Ku              265
Shinagawa Ku            259
Chiyoda Ku              257
Koto Ku                 233
Adachi Ku               224
Bunkyo Ku               154
Nerima Ku                93
Meguro Ku                91
Hachioji Shi             51
Machida Shi              38
Hino Shi                 26
Nishitokyo Shi           21
Musashino Shi            18
Mitaka Shi               16
Chofu Shi                16
Oshima Machi             15
Higashimurayama Shi      15
Kokubunji Shi            13
Akiruno Shi              13
Kodaira Shi              12
Kunitachi Shi       

## Filtered to Shinjuku District and counted minimum nights

In [146]:
grouped1 = df_listings.groupby(['neighbourhood'])['minimum_nights']

grouped1.value_counts().loc['Shinjuku Ku']


minimum_nights
1     1624
2      783
3      122
30     103
7       31
5       20
14      14
15      13
28      13
23       9
29       8
10       7
31       6
4        5
13       3
12       2
21       2
50       2
20       1
32       1
99       1
Name: minimum_nights, dtype: int64

In [271]:
df_listings.groupby(['neighbourhood', 'minimum_nights']).count().loc['Shinjuku Ku']

,id,name,host_id,host_name,neighbourhood_group,latitude,longitude,room_type,price,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,US_Price
minimum_nights,,,,,,,,,,,,,,,
1,1624,1624,1624,1617,0,1624,1624,1624,1624,1624,1303,1303,1624,1624,1624
2,783,783,783,783,0,783,783,783,783,783,678,678,783,783,783
3,122,122,122,122,0,122,122,122,122,122,94,94,122,122,122
4,5,5,5,5,0,5,5,5,5,5,4,4,5,5,5
5,20,20,20,20,0,20,20,20,20,20,17,17,20,20,20
7,31,31,31,31,0,31,31,31,31,31,25,25,31,31,31
10,7,7,7,7,0,7,7,7,7,7,5,5,7,7,7
12,2,2,2,2,0,2,2,2,2,2,0,0,2,2,2
13,3,3,3,3,0,3,3,3,3,3,1,1,3,3,3


# Which is the most popular district in Tokyo, Japan on Airbnb?

Shinjuku district.


# Why is that district popular?

Shinjuku district is equivalent to Times Sq in NY. It is the heart of Tokyo and can be reached to every district within short commute time by train.


# Average price per room type in Shinjuku District

In [253]:
#Average price in Japanese Yen
grouped2 = df_listings.groupby(['neighbourhood', 'room_type'])['price'].mean().round(2)

grouped2.loc['Shinjuku Ku']

room_type
Entire home/apt    24294.04
Hotel room         10389.41
Private room       15418.29
Shared room         3348.53
Name: price, dtype: float64

In [280]:
#Average price converted to USD
grouped2a = df_listings.groupby(['neighbourhood', 'room_type'])['US_Price'].mean().round(2)

grouped2a.loc['Shibuya Ku']



room_type
Entire home/apt    286.25
Hotel room          94.97
Private room       248.17
Shared room         53.61
Name: US_Price, dtype: float64

# Added new column to convert from Japanese Yen to USD

In [250]:
df_listings['US_Price'] = df_listings['price']*0.0095

df_listings['US_Price'] = df_listings['US_Price'].round(2)

df_listings

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,US_Price
0,35303,"La Casa Gaienmae C Harajuku, Omotesando is nearby",151977,Miyuki,NaN,Shibuya Ku,35.67152,139.71203,Private room,4192,28,18,2018-07-28,0.18,3,89,39.82
1,197677,Oshiage Holiday Apartment,964081,Yoshimi & Marek,NaN,Sumida Ku,35.71721,139.82596,Entire home/apt,10965,3,165,2020-03-04,1.56,1,270,104.17
2,289597,Private apt in central Tokyo #203,341577,Hide&Kei,NaN,Nerima Ku,35.74267,139.65810,Entire home/apt,4192,28,113,2020-02-17,1.17,2,242,39.82
3,370759,"Cozy flat #203, local area YET 10 mins to shib...",1573631,"Gilles,Mayumi,Taiki",NaN,Setagaya Ku,35.66443,139.65707,Entire home/apt,6987,28,103,2020-04-16,1.37,3,364,66.38
4,700253,Private apt in central Tokyo #201,341577,Hide&Kei,NaN,Nerima Ku,35.74264,139.65832,Entire home/apt,3977,28,104,2020-05-12,1.12,2,353,37.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15004,43571229,★Sale★Ueno/Ginza/Tokyo sta./Asakusa/Airport※6~...,29966538,Ostay-Jennie,NaN,Taito Ku,35.70971,139.77973,Entire home/apt,49987,7,0,NaN,NaN,49,80,474.88
15005,43571552,★Sale★Ueno/Ginza/Tokyo sta./Asakusa/Airport※6~...,230616333,Ostay-Ryu,NaN,Taito Ku,35.71042,139.77997,Entire home/apt,49987,7,0,NaN,NaN,13,80,474.88
15006,43572604,【ﾃﾚﾜｰｸ利用可】高速Wi-Fi･ﾜｰｸﾃﾞｽｸ完備･渋谷駅徒歩8分100平米戸建･駐車場,53446028,Ko,NaN,Shibuya Ku,35.65446,139.70728,Entire home/apt,49772,2,0,NaN,NaN,1,339,472.83
15007,43572959,Uhome高田马场ｱﾊﾟｰﾄﾒﾝﾄ/最大4名/新宿4分/池袋4分/銀座31分 TB2 301,73463626,Hara,NaN,Shinjuku Ku,35.70936,139.69962,Entire home/apt,12040,1,0,NaN,NaN,82,363,114.38


In [281]:
pd.merge(df_listings, df_calendar, on='listing_id')

KeyError: 'listing_id'

In [231]:
shinjuku_district = df_listings['neighbourhood'] =='Shinjuku Ku'

df_listings.loc[shinjuku_district, ['room_type', 'price']]

,room_type,price
6,Entire home/apt,5160
9,Private room,10750
16,Entire home/apt,27950
21,Private room,6020
22,Private room,6020
...,...,...
14977,Entire home/apt,12040
14978,Private room,12577
15000,Entire home/apt,9997
15001,Entire home/apt,12040


In [232]:
df_listings

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,35303,"La Casa Gaienmae C Harajuku, Omotesando is nearby",151977,Miyuki,NaN,Shibuya Ku,35.67152,139.71203,Private room,4192,28,18,2018-07-28,0.18,3,89
1,197677,Oshiage Holiday Apartment,964081,Yoshimi & Marek,NaN,Sumida Ku,35.71721,139.82596,Entire home/apt,10965,3,165,2020-03-04,1.56,1,270
2,289597,Private apt in central Tokyo #203,341577,Hide&Kei,NaN,Nerima Ku,35.74267,139.65810,Entire home/apt,4192,28,113,2020-02-17,1.17,2,242
3,370759,"Cozy flat #203, local area YET 10 mins to shib...",1573631,"Gilles,Mayumi,Taiki",NaN,Setagaya Ku,35.66443,139.65707,Entire home/apt,6987,28,103,2020-04-16,1.37,3,364
4,700253,Private apt in central Tokyo #201,341577,Hide&Kei,NaN,Nerima Ku,35.74264,139.65832,Entire home/apt,3977,28,104,2020-05-12,1.12,2,353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15004,43571229,★Sale★Ueno/Ginza/Tokyo sta./Asakusa/Airport※6~...,29966538,Ostay-Jennie,NaN,Taito Ku,35.70971,139.77973,Entire home/apt,49987,7,0,NaN,NaN,49,80
15005,43571552,★Sale★Ueno/Ginza/Tokyo sta./Asakusa/Airport※6~...,230616333,Ostay-Ryu,NaN,Taito Ku,35.71042,139.77997,Entire home/apt,49987,7,0,NaN,NaN,13,80
15006,43572604,【ﾃﾚﾜｰｸ利用可】高速Wi-Fi･ﾜｰｸﾃﾞｽｸ完備･渋谷駅徒歩8分100平米戸建･駐車場,53446028,Ko,NaN,Shibuya Ku,35.65446,139.70728,Entire home/apt,49772,2,0,NaN,NaN,1,339
15007,43572959,Uhome高田马场ｱﾊﾟｰﾄﾒﾝﾄ/最大4名/新宿4分/池袋4分/銀座31分 TB2 301,73463626,Hara,NaN,Shinjuku Ku,35.70936,139.69962,Entire home/apt,12040,1,0,NaN,NaN,82,363
